# Automated Anomaly Detection
**Objective**: Understand and practice automated anomaly detection using various techniques.

**Task**: Autoencoders for Anomaly Detection

**Steps**:
1. Data Set: Download a dataset of electricity consumption data.
2. Build an Autoencoder: Construct a simple autoencoder using a neural network for the
normal consumption data.
3. Identify Anomalies: Use the trained model to reconstruct the data and identify anomalies based on reconstruction error.
4. Visualize: Plot both the actual and reconstructed data to highlight anomalies.

In [5]:
# write your code from here
import unittest
import numpy as np

class TestAnomalyDetection(unittest.TestCase):

    def test_generate_data(self):
        df = generate_data()
        self.assertEqual(len(df), 1010)  # 1000 normal + 10 anomalies
        self.assertFalse(df['consumption'].isnull().any())

    def test_build_autoencoder(self):
        model = build_autoencoder(1)
        self.assertIsNotNone(model)
        self.assertEqual(model.input_shape, (None, 1))

    def test_detect_anomalies_with_autoencoder(self):
        df = generate_data()
        df, threshold = detect_anomalies_with_autoencoder(df)
        self.assertIn('reconstruction_error', df.columns)
        self.assertIn('is_anomaly', df.columns)
        self.assertTrue(df['is_anomaly'].dtype == bool or df['is_anomaly'].dtype == np.bool_)

if __name__ == "__main__":
    unittest.main(argv=[''], verbosity=2, exit=False)


test_build_autoencoder (__main__.TestAnomalyDetection) ... ok
test_detect_anomalies_with_autoencoder (__main__.TestAnomalyDetection) ... 

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


ok
test_generate_data (__main__.TestAnomalyDetection) ... ok

----------------------------------------------------------------------
Ran 3 tests in 4.989s

OK
